# Configuracion y conexion al servidor

## ¡No modificar la siguiente celda de codigo!

La siguiente celda es la que se encarga de conectar el notebook con la base de datos del curso de forma que se puedan hacer consultas de SQL en celdas de Python. Si se modifica, probablemente no funcionen los ejemplos y ejercicios de este notebook.

In [11]:
%load_ext sql
connection_string = f"postgresql://cc3201:j'<3_cc3201@cc3201.dcc.uchile.cl:5440/td3201"
%sql $connection_string

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


'Connected: cc3201@td3201'

---

# SQL: Gestionar, crear y actualizar tablas

Para poder trabajar con un esquema en SQL es necesario saber crear uno desde cero. A continuacion veremos como crear y definir todo lo que necesitamos para hacer un esquema en SQL. En este notebook crearemos una replica del esquema del sistema solar con el que hemos estado trabajando hasta ahora.

Para empezar, ejecuta el siguiente cuadro de codigo:

In [ ]:
%%sql
DROP SCHEMA SistemaSolar2 CASCADE;

Nota: al terminar de usar este laboratorio, se recomienda correr de nuevo el bloque de codigo anterior.

## Esquemas

Lo primero que se debe hacer es crear el esquema en la base de datos. Un esquema como ya deben saberlo, es una agrupacion de tablas que suelen estar relacionadas entre si. Para crearlos se usa la siguiente sintaxis:

In [ ]:
%%sql
CREATE SCHEMA SistemaSolar2;

Ya con esto esta creado el nuevo esquema, que llamaremos SistemaSolar2.

### Privilegios de Esquemas

Para poder modificar un esquema, debes tener los permisos (tambien llamados privilegios). Existen distintos niveles de privilegios en un esquema, pueden ser solo de lectura, pasando por lectura y escritura, hasta todos los privilegios que puedan haber para un esquema. Si desea saber mas sobre esto puede leer la documentacion de Postgres en el siguiente link (en ingles): https://www.postgresql.org/docs/13/ddl-priv.html 

La sintaxis para conceder privilegios es la siguiente:

```sql
GRANT [Nivel de Privilegios] ON SCHEMA [Esquema] TO [Usuario];
```

De esta forma, completando lo anterior, se pueden otorgar los permisos para un usuario en SQL.

### Search Path

Cuando uno define un esquema, se puede agregar al search_path del sistema. El search path puede contener varios esquemas, y el que se menciona primero es el que tendra prioridad. La sintaxis es la siguiente:

```sql
SET search_path TO [Esquema 1], [Esquema 2], ..., public;
```

Luego de definir todos los esquemas, se debe incluir el esquema 'public'. Una recomendacion es que antes de definir el searchpath se verifique cual es el search actual y que se agregue el esquema en la posicion que se desea a ese searchpath. Para ver cual es el search path se usa: 

```sql
SHOW search_path;
```

La principal funcion del search path es definir en que orden se buscan las tablas que se piden en las consultas. Si tuvieras un search path que fuera "SistemaSolar, Public" y existieran las tablas Sistemasolar.Aterrizaje y public.Aterrizaje, si se hace una consulta a la tabla Aterrizjae sin especificar el esquema se eligira siempre la perteneciente a SistemaSolar.


## Crear Tablas

Ya teniendo el esquema creado, y con los privilegios otorgados a los usuarios, procederemos a ver como se crean las tablas.

Para crear una tabla, se tiene que usar la siguiente sintaxis:

```sql
CREATE TABLE [Esquema].[Nombre de la tabla](
    [nombre de la columna] [Datatype de la columna],
    ...
)
```

Es decir, luego de especificar el nombre de la tabla y el esquema al que pertenece, se deben listar todos los nombres de las columnas seguidos del tipo de dato que contiene dicha columna. Pueden ver una lista de algunos tipos de datos en el siguiente link: https://www.ibiblio.org/pub/linux/docs/LuCaS/Tutoriales/NOTAS-CURSO-BBDD/notas-curso-BD/node134.html 

Ya vista la sintaxis, veamos un ejemplo de como crear tablas en nuestro esquema SistemaSolar2:

In [ ]:
%%sql

CREATE TABLE SistemaSolar2.Aterrizaje (
    nave VARCHAR(255),
    planeta VARCHAR(255),
    pais VARCHAR(255),
    anho SMALLINT
);

## Borrar tablas

Al igual que creamos tablas, podemos eliminarlas. La sintaxis para esto seria:
```sql 
DROP TABLE [esquema].[nombre de la tabla]; 
```

Eliminemos ahora la tabla que acabamos de crear:

In [ ]:
DROP TABLE Sistemasolar2.Aterrizaje

## Insertar Tuplas

Ahora veamos como agregar filas a una tabla. Para esto necesitamos una tabla en nuestro esquema, asi que cremos la tabla Aterrizaje de nuevo:

In [ ]:
%sql

CREATE TABLE SistemaSolar2.Aterrizaje (
    nave VARCHAR(255),
    planeta VARCHAR(255),
    pais VARCHAR(255),
    anho SMALLINT
);

Ya teniendo la tabla, podemos proceder a agregar las tuplas:

In [ ]:
%%sql

INSERT INTO sistemasolar2.aterrizaje VALUES ('Messenger','Mercurio','EEUU',2015);
INSERT INTO sistemasolar2.aterrizaje VALUES ('Venera 3','Venus','URRS',1966);
INSERT INTO sistemasolar2.aterrizaje VALUES ('Pioneer','Venus','EEUU',1978);

Es importante destacar que los datos a ingresar deben corresponder con el tipo de datos de la columna. Si fueramos a agregar la siguiente consulta:
```sql
INSERT INTO sistemasolar2.aterrizaje VALUES ('Mars 2 lander','Marte','URRS','1971');
```
nos devolveria un error, pues el tipo de datos de la 4ta columna (anho) es SMALLINT y estamos intentando agregar un STRING.

### Agregar Nulos

En capitulos anteriores definimos que un nulo es un valor desconocido, por lo que no se debe agregar nada a esa columna. La forma de hacerlo es la siguiente:

In [ ]:
INSERT INTO sistemasolar2.aterrizaje (pais, nave, planeta) VALUES ('EEUU','Mars 2 Lander','Marte');

Como no se especificó ningun valor para la columna 'anho', entonces se asigna un nulo a esa columna. Ademas, al especificar las columnas a las que se le van a agregar los datos, ya no importa el orden de las columnas en la tabla. Veamos como se ve la tabla Aterrizaje luego de haber agregado las filas anteriores:

In [ ]:
%%sql

SELECT * 
FROM sistemasolar2.aterrizaje;

### Insertar Tuplas desde otra tabla

Otra forma de agregar tuplas a una tabla es importarlas desde otra. Para esto crearemos una tabla en la que pondremos todos los aterrizajes realizados por EEUU. Primero creamos la tabla:

In [ ]:
%sql

CREATE TABLE SistemaSolar2.AterrizajeEEUU (
    nave VARCHAR(255),
    planeta VARCHAR(255),
    pais VARCHAR(255),
    anho SMALLINT
);

Ahora, insertaremos todas las tuplas que queremos. Para esto usaremos algo similar a una consulta anidada en el INSERT:

In [ ]:
%%sql

INSERT INTO AterrizajeEEUU ( SELECT * FROM Aterrizaje WHERE pais='EEUU');

Y ahora veamos como quedo dicha tabla:

In [ ]:
%%sql

SELECT *
FROM AterrizajeEEUU;

## Editar Tuplas

Ya teniendo una tabla con contenido, de ser necesario podemos editar una tupla. La sintaxis seria la siguiente:

```sql
UPDATE [esquema].[tabla]
SET [columna1]=[nuevo valor], ...
WHERE [condicion];
```

Como podran notar, se pueden cambiar el valor de varias columnas a la vez. Sin embargo, si solo se quiere cambiar de una tambien es posible. La manera de definir esto es usando la condicion, si queremos una sola se usa la llave primaria, y de ser varias simplemente se elige que atributos tienen en comun en la condicion.

Por ejemplo, queremos cambiar en la tabla Aterrizaje el pais y el año del Mars 2 Lander, entonces hariamos lo siguiente:

In [ ]:
%%sql

UPDATE sistemasolar2.Aterrizaje
SET anho=1971, pais='URSS'
WHERE nave='Mars 2 lander';

Con esto se actualizaria la tupla como podemos ver a continuacion:

In [ ]:
%%sql

SELECT * 
FROM sistemasolar2.Aterrizaje;

Y es importante mencionar que, aunque la tabla AterrizajeEEUU se creo con los datos de la tabla Aterrizaje, esta es una tabla independiente por lo que no se vera alterada aunque se modifiquen en Aterrizaje las columnas que tiene en común.

In [ ]:
%%sql

SELECT *
FROM sistemasolar2.AterrizajeEEUU;

## Actualizar tuplas de otra tabla

Si existen cambios en una tabla, podemos hacer que estos cambios se actualicen, de forma que la informacion de las tablas del esquema sea consistente entre si. Para esto usaremos la siguiente sintaxis:

```sql
UPDATE [Esquema].[Tabla por actualizar] [Alias1]
SET [Alias1].[Valor por actualizar] = [Alias2].[Valor actualizado]
FROM [Esquema].[Tabla actualizada]
WHERE [condicion/es];
```

Digamos que queremos asegurarnos que los aterrizajes hechos por EEUU estan correctos en la tabla Aterrizaje. Para esto actualizaremos esta columna desde los valores de AterrizajeEEUU:

In [ ]:
%%sql

UPDATE sistemasolar2.Aterrizaje A
SET A.pais=AE.pais
FROM AterrizajeEEUU AE
WHERE A.nave=AE.nave AND A.planeta=AE.planeta;

De esta forma el valor del pais en la fila correspondiente al Mars 2 lander cambia de URSS a EEUU:

In [ ]:
%%sql

SELECT *
FROM sistemasolar2.Aterrizaje;

## Borrar Tuplas


Si asi lo queremos, podemos borrar tuplas basandonos en condiciones de la siguiente manera:
```sql
DELETE FROM [esquema].[tabla] WHERE [Condicion]
```
Ejemplo:

In [ ]:
%%sql
DELETE FROM sistemasolar2.AterrizajeEEUU WHERE anho IS NULL;

## Borrar Columnas

Tambien podemos borrar columnas de una tabla. Para esto simplemente se necesita el nombre de la tabla y la columna que se quiere eliminar:

```sql
ALTER TABLE [esquema].[tabla] DROP COLUMN [Nombre de la columna];
```

Como ejemplo eliminaremos la columna pais de la tabla AterrizajeEEUU:

In [ ]:
%%sql
ALTER TABLE sistemasolar2.AterrizajeEEUU DROP COLUMN pais;

## Crear Columnas

Similarmente, se puede agregar una columna especificando la tabla a la que se quiere agregar, el nombre de la nueva columna y el tipo de dato:

```sql
ALTER TABLE [esquema].[Tabla] ADD COLUMN [nombre de la columna] [Tipo de datos];
```

Agreguemos una columna que tenga la fecha del despegue en AterrizajeEEUU:

In [ ]:
%%sql

ALTER TABLE sistemasolar2.AterrizajeEEUU ADD COLUMN despegue DATE;

## Modificar Columnas

En una columna, podemos modificar el tipo de datos que contiene. A continuacion la sintaxis:
```sql
ALTER TABLE [esquema].[tabla] ALTER COLUMN [Nombre de la columna] [Nuevo tipo de datos];
```

Ahora modificaremos el tipo de datos en la columna despegue de la tabla AterrizajeEEUU:

In [ ]:
ALTER TABLE sistemasolar2.AterrizajeEEUU ALTER COLUMN despegue VARCHAR(255);

## Cargar Datos

Para terminar con la seccion de creacion y actualizacion de tablas, vamos a ver como cargar los datos desde un archivo a una tabla ya preexistente. Para esto se usa la siguiente sintaxis:

```sql
COPY [Nombre de la tabla] FROM ['Direccion del archivo'] DELIMITER [Separador de las columnas];
```

Por ejemplo, digamos que tenemos los datos en un .csv (coma separated values), lo que significa que cada linea del archivo es una fila y cada columna esta separada por comas. Entonces el comando quedaria de la siguiente manera:

```sql
COPY Aterrizaje FROM '/home/ahogan/documents/aterrizaje.csv' DELIMITER ',';
```

Esta forma de agregar datos es especifica de Postgres, y no sobreescribe las filas ya existentes, sino que concatena las nuevas. Es decir, todas las tuplas nuevas que se agreguen iran despues de las que ya existian en la tabla.

# Restricciones

Al comienzo del curso se vio el concepto de llaves primarias y foraneas. Estas son algunas de las restricciones que se deben definir al momento de crear una tabla. Una restriccion se puede definir como una restriccion formal que se le impone a un esquema y que todas sus instancias deben satisfacer.

Para esta parte del laboratorio vamos a usar un esquema relacional distinto al que estabamos usando. Ahora estaremos trabajando con el esquema relacional 'Banco'. Si desean leer mas pueden revisar el READ ME.

**Me gustaria poner una foto aqui con todas las tablas para que se pueda ver el esquema mas facilmente**

## Restricciones Basicas

Partamos con las restricciones mas basicas que puede tener un esquema:

### Llaves, Nulos, Dominio

- Dominio: La restriccion de dominio ya la habiamos visto antes, pues con ella nos referimos a definir el tipo de dato (BIGINT, VARCHAR, etc.) luego de declarar el nombre de la columna.

- Llaves: Toda tabla debe tener al menos una llave primaria. La forma de agregar una llave primaria es escribir el nombre de la columna, el tipo de datos (como lo vimos en la parte de crear una tabla) y luego colocar 'PRIMARY KEY' antes de la siguiente columna.

- Nulos: A veces necesitamos definir que un valor no pueda ser nulo, o desconocido. Llevandolo a nuestro ejemplo, un banco no puede no saber cual es el saldo de una cuenta o el rut de un cliente. En estos casos se debe agregar 'NOT NULL' luego del nombre y tipo de datos al crear la tabla. 

Por defecto, las llaves nunca pueden ser nulos, por lo que si se coloca 'PRIMARY KEY' no se debe colocar 'NOT NULL'

Ahora veamos como se aplicaria esto a la creacion de la tabla Cliente en el esquema banco. El comando quedaria de la siguiente forma:

```sql
CREATE TABLE banco.cliente (
    rut VARCHAR(12) NOT NULL PRIMARY KEY,
    nombre VARCHAR(255) NOT NULL,
    fono BIGINT NOT NULL,
    direccion VARCHAR(255) NOT NULL
)
```

### Valores por defecto

Al crear una tabla tambien tenemos la opcion de definir un valor por defecto, de forma que si no se especifica este valor al momento de crear la tupla no se guarde como nulo sino como otra cosa. La forma de hacerlo seria usando el termino DEFAULT y luego colocar el valor por defecto.

Es importante destacar que DEFAULT, a diferencia de PRIMARY KEY, no funciona como un NOT NULL implícito. Es decir, aunque no se puede crear una tupla con un valor nulo, despues de creado si se podria cambiar este valor a un nulo. Por lo tanto, se debe especificar si la columna no puede ser nulo independiente de si se define un valor por defecto.

Ahora veamos como hariamos la tabla Cuenta. Ademas de las restricciones de llaves y nulos, el saldo_clp y el saldo_usd deben partir en cero ya que seria como que no tienen dinero en la cuenta. Para esto el comando para crear la cuenta quedaria de la siguiente manera:

```sql
CREATE TABLE Banco.Cuenta (
    numero BIGINT PRIMARY KEY,
    rut VARCHAR(12) NOT NULL,
    tipo VARCHAR(12) NOT NULL,
    saldo_clp BIGINT NOT NULL DEFAULT 0,
    saldo_usd FLOAT NOT NULL DEFAULT 0
)
```

## Restricciones de Unicidad

A veces no solo la llave primaria es la unica que no debe repetirse. Por ejemplo, en nuestro banco no podrian existir dos tuplas con el mismo rut y tipo de cuenta, ya que cada cliente solo puede tener una cuenta de cada tipo pues tendria sentido que un cliente tuviese dos cuentas corrientes.

Para esto se usa la restriccion de unicidad. La unicidad se usa para todas las llaves candidatas. Y pueden exstir varias llaves candidatas, pero solo una llave primaria. 

Para declararla se deben definir todas las columnas con sis restricciones respectivas, y al final agregar 'UNIQUE ([...)' especificando entre parentesis cuales columnas deben ser unicas, separandolas por comas. 

Si agregamos esta restriccion de Unicidad a Cuenta, ahora quedaria de la siguiente manera:

```sql
CREATE TABLE Banco.Cuenta (
    numero BIGINT PRIMARY KEY,
    rut VARCHAR(12) NOT NULL,
    tipo VARCHAR(12) NOT NULL,
    saldo_clp BIGINT NOT NULL DEFAULT 0,
    saldo_usd FLOAT NOT NULL DEFAULT 0,
    UNIQUE (rut,tipo)
)
```

Nota: PRIMARY KEY implica unicidad, por lo que no es necesario especificar unicidad en cuanto a la llave primaria se refiere.


## Restricciones de llaves Foraneas

Ademas de las llaves primarias, tenemos tambien las llaves foraneas. Para declarar una llave foranea primero se escribe el nombre de la columna y el tipo de datos, seguido de 'REFERENCES TO' y por ultimo la columna a la que se referencia. Al hacer esto no se podran agregar valores a esta columna que no esten en la columna a la que se hace referencia.

Ahora agreguemos una referencia a la tabla Cuenta. Si la tabla Cliente contiene la lista de todos los clientes, entonces no puede existir una cuenta con un rut que no este en la lista de clientes. Por lo tanto agregamos esta referencia:


```sql
CREATE TABLE Banco.Cuenta (
    numero BIGINT PRIMARY KEY,
    rut VARCHAR(12) REFERENCES TO cliente(rut),
    tipo VARCHAR(12) NOT NULL,
    saldo_clp BIGINT NOT NULL DEFAULT 0,
    saldo_usd FLOAT NOT NULL DEFAULT 0,
    UNIQUE (rut,tipo)
)
```

Si comparamos esta consulta con la anterior, podemos ver que quitamos el NOT NULL. Esto debido a que en una columna que referencia a otra no pueden haber valores desconocidos, por lo tanto REFERENCES TO incluye un NOT NULL implicito.



Si existen llaves foraneas, no se podra hacer DROP TABLE de las tablas a las que se referencia. Es decir, en este caso ya no se podria ejecutar la siguiente linea:

```sql
DROP TABLE Banco.Cliente;
```

Para poder eliminar esta tabla se debe usar CASCADE al final del comando, y de esta forma se eliminaria la tabla Cliente a la vez que se elimina la restriccion de llave foranea de la tabla Cuenta. Se veria de la siguiente manera:

```sql
DROP TABLE Banco.Cliente CASCADE;
```


## Restricciones de llaves compuestas

Hasta ahora hemos visto como hacer referencias a una llave que sea de un solo atributo. Sin embargo pueden existir llaves compuestas por mas de una columna, y para esto se debe usar una sintaxis distinta. 

Para referenciar una llave compuesta se debe especificar al final luego de definir todos los nombres, tipos de datos y restriccion de nulos de las columnas. Si es una llave primaria se coloca PRIMARY KEY seguido de las columnas que componen la llave entre parentesis y separadas por coma. Veamos un ejemplo con la tabla Divisa:

```sql
CREATE TABLE banco.divisa (
    d1 VARCHAR(3),
    d2 VARCHAR(3),
    valor DOUBLE PRECISION,
    PRIMARY KEY (d1,d2)
)
```

Si lo que queremos es definir una llave foranea, se debe escribir FOREIGN KEY seguido de la tupla de columnas igual que en la llave primaria. Luego se coloca REFERENCES y se especifica la tabla y las columnas a las que se referencia respectivamente. Veamos como seria en la tabla Cambio:

```sql
CREATE TABLE banco.cambio (
    id VARCHAR(12),
    venta VARCHAR(3),
    compra VARCHAR(3),
    monto DOUBLE PRECISION,
    FOREIGN KEY (venta,compra) REFERENCES Divisa (d1,d2),
    PRIMARY KEY (id, venta, compra)
)
```


## Nombrar (y borrar) restricciones

Lo ultimo que veremos de restricciones sera como agregarlas luego de que ya se han creado las tablas. Para esto es necesario introducir el concepto de nombrar las restricciones.

Al crear una restriccion, se le puede asignar un nombre o un alias. Esto es posible solo si se usa una sintaxis como la de las llaves compuestas o la de la unicidad. Es decir, usando PRIMARY KEY al final y no declarandolo en la misma linea de la columna. Para agregar un alias se debe incluir 'CONTRAINT [Alias]' antes de declarar el tipo de restriccion que se va a usar. Veamoslo en la tabla Cuenta, donde le daremos un nombre a la llave primaria y a la restriccion de unicidad:

```sql
CREATE TABLE Banco.Cuenta (
    numero BIGINT,
    rut VARCHAR(12) REFERENCES cliente(rut),
    tipo VARCHAR(12) NOT NULL,
    saldo_clp BIGINT NOT NULL DEFAULT 0,
    saldo_usd FLOAT NOT NULL DEFAULT 0,
    CONSTRAINT Cuenta_uni_rt UNIQUE (rut,tipo),
    CONSTRAINT Cuenta_pk PRIMARY KEY (numero)
)
```


De esta forma, cuando queramos referirnos a la llave primaria podemos usar el alias 'Cuenta_pk'. Esto es util para lo que veremos a continuacion.

Si por algun motivo quisieramos eliminar las restricciones de llave primaria o de unicidad de la tabla cuenta, podremos hacerlo con la siguiente sintaxis:

```sql
ALTER TABLE [esquema].[tabla] DROP CONSTRAINT [Alias de la restriccion];
```

Por ejemplo si queremos quitar la restriccion de Unicidad hariamos lo siguiente:

```sql
ALTER TABLE Banco.cuenta DROP CONSTRAINT Cuenta_uni_rt;
```

De la misma manera, podemos agregar restricciones a una tabla que ya existe. Para esto se usa la siguiente sintaxis:

```sql
ALTER TABLE [esquema].[tabla] ADD CONSTRAINT [Alias de la restriccion] [Definicion de la restriccion];
```

Si ahora quisieramos agregar de nuevo la restriccion de Unicidad que eliminamos antes, pero con otro nombre, podriamos hacer lo siguiente:

```sql
ALTER TABLE Banco.cuenta ADD CONSTRAINT Unicidad_rut_tipo UNIQUE (rut,tipo);
```

Y de esta forma se habria eliminado la restriccion de unicidad, y agregado la misma restriccion de nuevo pero con otro nombre.

# Referencias

- https://datademia.es/blog/que-es-sql

- Capítulo 5 Database Management Systems, Ramakrishnan / Gehrke (Third Edition)

- https://www.w3resource.com/sql/joins/sql-joins.php

- https://www.postgresql.org/docs/13/ddl-priv.html

- https://www.ibiblio.org/pub/linux/docs/LuCaS/Tutoriales/NOTAS-CURSO-BBDD/notas-curso-BD/node134.html